In [2]:
import os
import glob
import numpy as np
import pandas as pd
import torch
import SimpleITK as sitk
from tqdm import tqdm
from monai.networks.nets import BasicUNet
from monai.losses import DiceCELoss
from monai.data import Dataset, DataLoader
from monai.transforms import (
    Compose, EnsureChannelFirstd, ScaleIntensityd, ToTensord, ResizeWithPadOrCropd
)
from torch.nn.functional import one_hot
from torch.optim import Adam

# ==== USER PATHS ====
KTRANS_DIR = r"C:\Users\anude\Downloads\ProstateXKtrains-train-fixed"
DICOM_ROOT = r"C:\Users\anude\Downloads\project"
FINDINGS_CSV = r"C:\Users\anude\Downloads\project\ProstateX-Findings-Train100.csv"
IMAGES_CSV = r"C:\Users\anude\Downloads\project\ProstateX-Images-Train100.csv"


In [3]:
# ==== READ CSVs ====
print("Loading metadata CSVs...")
findings_df = pd.read_csv(FINDINGS_CSV)
images_df = pd.read_csv(IMAGES_CSV)
meta_df = pd.merge(images_df, findings_df, on=["ProxID", "fid"])
print("Metadata loaded and merged.")


Loading metadata CSVs...
Metadata loaded and merged.


In [4]:
# ==== UTILS ====
def load_dicom_volume(proxid, series_description="ADC"):
    patient_dir = os.path.join(DICOM_ROOT, proxid)
    found_dicom_files = []
    for root, dirs, files in os.walk(patient_dir):
        if series_description.lower() in root.lower():
            dicom_files = sorted(glob.glob(os.path.join(root, "*.dcm")))
            if dicom_files:
                found_dicom_files = dicom_files
                break
    if not found_dicom_files:
        raise RuntimeError(f"No DICOM series found for {proxid} matching '{series_description}'")
    reader = sitk.ImageSeriesReader()
    reader.SetFileNames(found_dicom_files)
    return reader.Execute()

def load_ktrans(proxid):
    ktrans_path = os.path.join(KTRANS_DIR, proxid, f"{proxid}-Ktrans.mhd")
    if not os.path.exists(ktrans_path):
        raise FileNotFoundError(f"Ktrans not found at {ktrans_path}")
    return sitk.ReadImage(ktrans_path)

def generate_label_mask(image, ijk_list):
    label = np.zeros(sitk.GetArrayFromImage(image).shape, dtype=np.uint8)
    for i, j, k in ijk_list:
        try:
            label[int(k), int(j), int(i)] = 1
        except:
            continue
    label_image = sitk.GetImageFromArray(label)
    label_image.CopyInformation(image)
    return label_image

def parse_ijk(ijk_str):
    try:
        parts = ijk_str.strip().replace(",", " ").split()
        if len(parts) != 3:
            raise ValueError("Invalid IJK format")
        i, j, k = map(int, parts)
        return i, j, k
    except Exception as e:
        print(f"\u26a0\ufe0f  Skipping bad ijk string: {ijk_str} — Reason: {e}")
        return None


In [5]:

# ==== CREATE OUTPUT FOLDERS ====
os.makedirs("images", exist_ok=True)
os.makedirs("labels", exist_ok=True)

In [6]:
# ==== BUILD DATA ====
data = []
print("\n📦 Building dataset...\n")
for proxid in tqdm(meta_df["ProxID"].unique()):
    try:
        print(f"\n🔍 Processing: {proxid}")
        adc_image = load_dicom_volume(proxid, series_description="ADC")
        print("✅ ADC loaded")

        ktrans_image = load_ktrans(proxid)
        if ktrans_image is None:
            raise ValueError("Missing Ktrans")

        ktrans_resampled = sitk.Resample(ktrans_image, adc_image)
        print("✅ Ktrans loaded and resampled")

        lesions = meta_df[meta_df["ProxID"] == proxid]
        ijk_coords = [
                        parse_ijk(ijk) for ijk in lesions["ijk"].dropna().tolist()
                     ]
        ijk_coords = [coord for coord in ijk_coords if coord is not None]
        if not ijk_coords:
            raise ValueError("No lesion ijk coordinates found")

        print(f"✅ Lesions found: {len(ijk_coords)}")
        label_image = generate_label_mask(adc_image, ijk_coords)

        # Save Nifti for MONAI
        adc_path = f"images/{proxid}_adc.nii.gz"
        ktrans_path = f"images/{proxid}_ktrans.nii.gz"
        label_path = f"labels/{proxid}_label.nii.gz"

        sitk.WriteImage(adc_image, adc_path)
        sitk.WriteImage(ktrans_resampled, ktrans_path)
        sitk.WriteImage(label_image, label_path)

        data.append({
            "image": [adc_path, ktrans_path],
            "label": label_path
        })
        print(f"✅ Added {proxid} to training set.")

    except Exception as e:
        print(f"❌ Failed on {proxid}: {str(e)}")

# ==== CHECK IF DATASET IS READY ====
print(f"\n✅ Total usable training samples: {len(data)}\n")
if len(data) == 0:
    raise ValueError("❌ No usable training data was found. Please check preprocessing steps.")


📦 Building dataset...



  0%|          | 0/101 [00:00<?, ?it/s]


🔍 Processing: ProstateX-0000


  1%|          | 1/101 [00:00<00:29,  3.39it/s]

✅ ADC loaded
✅ Ktrans loaded and resampled
✅ Lesions found: 9
✅ Added ProstateX-0000 to training set.

🔍 Processing: ProstateX-0001


...

✅ ADC loaded
✅ Ktrans loaded and resampled
✅ Lesions found: 12
✅ Added ProstateX-0098 to training set.

🔍 Processing: ProstateX-0099


 99%|█████████▉| 100/101 [00:24<00:00,  3.96it/s]

✅ ADC loaded
✅ Ktrans loaded and resampled
✅ Lesions found: 20
✅ Added ProstateX-0099 to training set.

🔍 Processing: ProstateX-0100


100%|██████████| 101/101 [00:24<00:00,  4.12it/s]

✅ ADC loaded
✅ Ktrans loaded and resampled
✅ Lesions found: 27
✅ Added ProstateX-0100 to training set.

✅ Total usable training samples: 101



In [7]:
"""# ==== MONAI TRANSFORMS ====
print("Setting up transforms...")
train_transforms = Compose([
    EnsureChannelFirstd(keys="label", channel_dim=0),
    ScaleIntensityd(keys="image"),
    ResizeWithPadOrCropd(keys=["image", "label"], spatial_size=(128, 128, 64)),
    ToTensord(keys=["image", "label"])
])

SyntaxError: incomplete input (3004016649.py, line 1)

In [11]:
from monai.transforms import LoadImaged

train_transforms = Compose([
    LoadImaged(keys=["image", "label"]),
    EnsureChannelFirstd(keys=["image", "label"]),
    ScaleIntensityd(keys="image"),
    ResizeWithPadOrCropd(keys=["image", "label"], spatial_size=(128, 128, 64)),
    ToTensord(keys=["image", "label"])
])


In [13]:
# ==== DATA LOADER ====
print("Initializing data loader...")
train_ds = Dataset(data=data, transform=train_transforms)
train_loader = DataLoader(train_ds, batch_size=1, shuffle=True)
print("Data loader ready.")

# ==== MODEL ====
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model = BasicUNet(spatial_dims=3, in_channels=2, out_channels=2).to(device)
loss_fn = DiceCELoss(to_onehot_y=True, softmax=True)
optimizer = Adam(model.parameters(), lr=1e-4)
print("Model initialized.")


Initializing data loader...
Data loader ready.
Using device: cuda
BasicUNet features: (32, 32, 64, 128, 256, 32).
Model initialized.


In [15]:
# ==== TRAINING ====
print("\n\ud83d\ude80 Starting training...\n")
for epoch in range(10):
    print(f"\n--- Epoch {epoch+1} ---")
    model.train()
    epoch_loss = 0
    for batch_idx, batch in enumerate(train_loader):
        print(f"\U0001F4CB Batch {batch_idx+1}")
        images = torch.cat(batch["image"], dim=1).to(device)
        labels = batch["label"].to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        print(f"Loss: {loss.item():.4f}")
        epoch_loss += loss.item()
    print(f"[Epoch {epoch+1}] Avg Loss: {epoch_loss:.4f}")

# ==== SAVE MODEL ====
torch.save(model.state_dict(), "prostate_seg_model.pth")
print("\n\u2705 Model saved as 'prostate_seg_model.pth'")


TypeError: cat() received an invalid combination of arguments - got (MetaTensor, dim=int), but expected one of:
 * (tuple of Tensors tensors, int dim = 0, *, Tensor out = None)
 * (tuple of Tensors tensors, name dim, *, Tensor out = None)


In [17]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


In [19]:
import time
import torch

print("\n🚀 Starting training...\n")
for epoch in range(10):
    print(f"\n--- Epoch {epoch + 1} ---")
    model.train()
    epoch_loss = 0
    start_time = time.time()

    for batch_idx, batch in enumerate(train_loader):
        images = batch["image"].to(device)
        labels = batch["label"].to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        # 🧹 Clear GPU memory after each batch
        torch.cuda.empty_cache()

        epoch_loss += loss.item()

        if (batch_idx + 1) % 10 == 0 or (batch_idx + 1) == len(train_loader):
            print(f"📦 Batch {batch_idx + 1}/{len(train_loader)} — Loss: {loss.item():.4f}")

    elapsed = time.time() - start_time
    print(f"✅ Epoch {epoch + 1} completed — Avg Loss: {epoch_loss:.4f} — ⏱ {elapsed:.2f} sec")

torch.save(model.state_dict(), "prostate_seg_model.pth")
print("\n✅ Model saved as 'prostate_seg_model.pth'")



🚀 Starting training...


--- Epoch 1 ---
📦 Batch 10/101 — Loss: 1.2813
📦 Batch 20/101 — Loss: 1.2290
📦 Batch 30/101 — Loss: 1.1769
📦 Batch 40/101 — Loss: 1.1301
📦 Batch 50/101 — Loss: 1.0783
📦 Batch 60/101 — Loss: 1.0384
📦 Batch 70/101 — Loss: 1.0195
📦 Batch 80/101 — Loss: 0.9974
📦 Batch 90/101 — Loss: 0.9791
📦 Batch 100/101 — Loss: 0.9726
📦 Batch 101/101 — Loss: 0.9706
✅ Epoch 1 completed — Avg Loss: 112.0996 — ⏱ 44.38 sec

--- Epoch 2 ---
📦 Batch 10/101 — Loss: 0.9611
📦 Batch 20/101 — Loss: 0.9421
📦 Batch 30/101 — Loss: 0.9358
📦 Batch 40/101 — Loss: 0.9175
📦 Batch 50/101 — Loss: 0.9087
📦 Batch 60/101 — Loss: 0.9031
📦 Batch 70/101 — Loss: 0.8956
📦 Batch 80/101 — Loss: 0.8840
📦 Batch 90/101 — Loss: 0.8798
📦 Batch 100/101 — Loss: 0.8673
📦 Batch 101/101 — Loss: 0.8695
✅ Epoch 2 completed — Avg Loss: 92.3045 — ⏱ 33.71 sec

--- Epoch 3 ---
📦 Batch 10/101 — Loss: 0.8581
📦 Batch 20/101 — Loss: 0.8487
📦 Batch 30/101 — Loss: 0.8429
📦 Batch 40/101 — Loss: 0.8379
📦 Batch 50/101 — Loss: 0.8289
📦

In [43]:
from monai.metrics import ConfusionMatrixMetric

metric = ConfusionMatrixMetric(include_background=True, metric_name="accuracy", reduction="mean")

# Convert to predicted class (argmax)
y_pred_labels = torch.argmax(outputs, dim=1, keepdim=True)  # Shape: [1, 1, D, H, W]

# Ensure label shape matches
if labels.shape[1] != 1:
    labels = labels[:, :1]

# Now metric will work
metric(y_pred=y_pred_labels, y=labels)
#accuracy = metric.aggregate().item()
accuracy = metric.aggregate()
accuracy_scalar = accuracy[0].item()  # Take the first value from the list
